In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.svm import SVC

from preprocessing import preprocesamiento_basico, normalizar_datos
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc,graficar_matriz_confusion
from funciones_auxiliares import traer_datasets, traer_dataset_prediccion_final, separar_dataset, encontrar_hiperparametros_RGSCV, mapear_target_binario

# Support Vector Machines

### Obtención de los datos y preprocesamiento

In [2]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)

X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_test.is_copy=False

y_train.set_index('id', inplace=True)
y_train = y_train.sort_values(by=['id'], ascending=True).copy()

y_test.set_index('id', inplace=True)
y_test = y_test.sort_values(by=['id'], ascending=True).copy()


Aplicamos metodo de feature engineering general

In [3]:
dataframes_procesados = preprocesamiento_basico([X_train, X_test])
X_train = dataframes_procesados[0]
X_test = dataframes_procesados[1]

Además es buena práctica realizar una normalización previa de las escalas de los datos, por lo que recurriremos al siguiente preprocesamiento adicional:

In [4]:
dataframes_normalizados = normalizar_datos([X_train, X_test])
X_train = dataframes_normalizados[0]
X_test = dataframes_normalizados[1]

In [5]:
X_train

array([[-0.11207163, -1.42769342,  0.11177061, ...,  0.67593366,
        -0.53062659, -0.10042639],
       [ 0.85991399, -1.28254569, -1.31112628, ..., -1.12909687,
        -0.53062659, -0.10042639],
       [ 1.30635422, -1.0406328 , -1.62732559, ...,  0.56311925,
        -0.53062659, -0.10042639],
       ...,
       [ 0.74192682, -1.33092826, -0.94222709, ..., -0.1137672 ,
        -0.53062659, -0.10042639],
       [ 0.92761988, -1.476076  , -0.67872766, ..., -0.1137672 ,
        -0.53062659, -0.10042639],
       [ 0.91512075, -1.33092826, -0.94222709, ..., -0.1137672 ,
        -0.53062659, -0.10042639]])

### Búsqueda de Hiperparámetros

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento. En el caso de SVM, estos serán 'C' (dureza ante outliers), 'kernel' (el kernel a utilizar para evitar problemas linealmente inseparables), 'degree' (grado del kernel polinomial, ignorado por los otros kernels) y  'max_iter' (el número máximo de iteraciones como criterio de paro).

In [6]:
params = {'C': [0.1, 1, 10], 'kernel': ['poly', 'rbf', 'sigmoid'], 'degree':[2,3,4], 'max_iter': [10000]}
hiperparametros = encontrar_hiperparametros_RGSCV(SVC(), params=params, x_np=X_train, y_np=y_train.to_numpy().ravel())

C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [9]:
c_elegido = hiperparametros['C']
max_iter_elegida = hiperparametros['max_iter']
grado_elegido = hiperparametros['degree']
kernel_elegido = hiperparametros['kernel']
print(f'Mejor C: {c_elegido}')
print(f'Mejor iteración máxima: {max_iter_elegida}')
print(f'Mejor kernel: {kernel_elegido}')
print(f'Mejor grado para kernel polinomial: {grado_elegido}')

Mejor C: 1
Mejor iteración máxima: 5000
Mejor kernel: sigmoid
Mejor grado para kernel polinomial: 4


### Entrenando el Modelo con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [11]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
    svm_clasificacion = SVC(max_iter=20000, C=c_elegido, kernel=kernel_elegido, degree=grado_elegido)
    svm_clasificacion.fit(X_train[train_index], y_train.iloc[train_index])
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], svm_clasificacion.predict(X_train[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Reporte para el FOLD 0
                                          precision    recall  f1-score   support

No llueven hamburguesas al dia siguiente       0.84      0.86      0.85     15891
   Llueven hamburguesas al dia siguiente       0.46      0.42      0.44      4581

                                accuracy                           0.76     20472
                               macro avg       0.65      0.64      0.64     20472
                            weighted avg       0.75      0.76      0.76     20472



C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Reporte para el FOLD 1
                                          precision    recall  f1-score   support

No llueven hamburguesas al dia siguiente       0.85      0.87      0.86     15891
   Llueven hamburguesas al dia siguiente       0.51      0.48      0.50      4580

                                accuracy                           0.78     20471
                               macro avg       0.68      0.68      0.68     20471
                            weighted avg       0.78      0.78      0.78     20471



C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Reporte para el FOLD 2
                                          precision    recall  f1-score   support

No llueven hamburguesas al dia siguiente       0.82      0.81      0.81     15891
   Llueven hamburguesas al dia siguiente       0.37      0.39      0.38      4580

                                accuracy                           0.71     20471
                               macro avg       0.60      0.60      0.60     20471
                            weighted avg       0.72      0.71      0.72     20471



C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Reporte para el FOLD 3
                                          precision    recall  f1-score   support

No llueven hamburguesas al dia siguiente       0.85      0.86      0.86     15891
   Llueven hamburguesas al dia siguiente       0.50      0.49      0.49      4580

                                accuracy                           0.77     20471
                               macro avg       0.68      0.67      0.67     20471
                            weighted avg       0.77      0.77      0.77     20471



C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Guillermo\anaconda3\envs\orgadatos\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Reporte para el FOLD 4
                                          precision    recall  f1-score   support

No llueven hamburguesas al dia siguiente       0.85      0.89      0.87     15890
   Llueven hamburguesas al dia siguiente       0.54      0.46      0.50      4581

                                accuracy                           0.79     20471
                               macro avg       0.69      0.67      0.68     20471
                            weighted avg       0.78      0.79      0.78     20471



### Predicción del modelo sobre holdout

Una vez entrenado y validado el modelo con CrossValidation, viendo que las métricas resultantes fueron razonables y no dan indicio de overfit o mala división del dataset, pasaremos a probar el modelo en la partición Holdout. Para esto usaremos los mejores hiperparámetros hallados previamente en Training.

In [1]:
svm_holdout = SVC(max_iter=50000, C=c_elegido, kernel=kernel_elegido, degree=grado_elegido, probability=True)
svm_holdout.fit(X_train, y_train.to_numpy().ravel())

NameError: name 'SVC' is not defined

In [ ]:
mapeo_binario_v = np.vectorize(mapear_target_binario)
y_pred = svm_holdout.predict(X_test)
y_pred_binario = mapeo_binario_v(y_pred)
y_pred_proba = svm_holdout.predict_proba(X_test)[:, 1]
y_test_binario = y_test['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0}).to_numpy()
print(classification_report(y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy(), y_pred))

### Curva AUC ROC


In [ ]:
graficar_auc_roc(y_test_binario, y_pred_proba)

### Matriz de confusion

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred_binario)

## Predicción con el dataset nuevo
A continuación, realizamos la predicción con el árbol sin expandir el dataset sobre el dataset de predicciones nuevo, y la escribimos al archivo 'SVM.csv'

In [ ]:
from funciones_auxiliares import exportar_prediccion_final

df_prediccion_final = traer_dataset_prediccion_final()
ids = df_prediccion_final['id'].to_numpy()

df_prediccion_final = preprocesamiento_basico([df_prediccion_final])[0]
df_preddiccion_final = normalizar_datos([df_prediccion_final])[0]
predicciones = svc_holdout.predict(df_prediccion_final.to_numpy())

exportar_prediccion_final(ids, predicciones, 'SVM.csv')